In [18]:
import pdfplumber
import pandas as pd
from src.widgets_helpers import file_selector_from_filder
import re



In [19]:
file = file_selector_from_filder("extractos/")

Dropdown(description='Archivos:', layout=Layout(width='50%'), options=(('Extracto_Febrero_2024.pdf', 'extracto…

In [20]:
file

'extractos/Extracto_Febrero_2024.pdf'

In [21]:
def extract_text_from_pdf(pdf_path):
    text = []
    with pdfplumber.open(pdf_path) as pdf:
        for pagina in pdf.pages:
            text.append(pagina.extract_text())
        return '\n'.join(text)

In [100]:
def extract_text_from_pdf(pdf_path):
    text = []
    with pdfplumber.open(pdf_path) as pdf:
        text.append(pdf.pages[0].extract_text())
        return '\n'.join(text)

In [101]:
text = extract_text_from_pdf(file)

In [108]:
# Función para separar las columnas en una línea de texto
def separar_columnas_cc(linea):
    partes = re.split(r'\s{2,}', linea)
    
    # Fechas y Ref
    fecha = partes[0].split()[0]
    ref = partes[0].split()[1]
    fecha_valor = partes[0].split()[2]
    
    rest_of_line = linea.replace(' '.join([fecha, ref, fecha_valor]), '').strip()
    
    
    # Descripción y valores numéricos
    # descripcion_partes = partes[1].split()
     # Inicializar variables para cargos, abonos y saldo
    saldo = rest_of_line.split()[-1]
    cargo_abono = rest_of_line.split()[-2]
    description = rest_of_line.replace(' '.join([cargo_abono, saldo]), '').strip()
    return [fecha, ref, fecha_valor, description,cargo_abono, saldo]

In [109]:
# Dividir el texto en líneas
lineas = text.split('\n')

# Filtrar las líneas relevantes
transacciones = []
for linea in lineas:
    # Asumir que cada línea relevante contiene una fecha en el formato dd-mm-yy
    if "-24 " in linea:  # Ajusta esto según tu formato de fecha
        transacciones.append(separar_columnas_cc(linea))

In [110]:
transacciones

[['02-02-24', '846', '01-02-24', 'RECIBO PLATINUM', '1.285,52', '5.793,33'],
 ['02-02-24',
  '006',
  '01-02-24',
  'PAGO BIZUM A NATALIA;GRANDA;AL',
  '4,40',
  '5.788,93'],
 ['07-02-24',
  '134',
  '07-02-24',
  'PAGO BIZUM A ROBERTO;PERINELLI',
  '85,00',
  '5.703,93'],
 ['10-02-24',
  '201',
  '10-02-24',
  'PAGO BIZUM A ANE;RAMSDEN;ARRIB',
  '33,00',
  '5.670,93'],
 ['10-02-24',
  '322',
  '10-02-24',
  'PAGO BIZUM A SAMIR RUBEN;AUYAN',
  '70,80',
  '5.600,13'],
 ['12-02-24',
  '042',
  '12-02-24',
  'PAGO BIZUM A DANIEL;TANO;LEAL',
  '5,00',
  '5.595,13'],
 ['12-02-24',
  '505',
  '11-02-24',
  'PAGO BIZUM A ANE;RAMSDEN;ARRIB',
  '5,00',
  '5.590,13'],
 ['12-02-24',
  '662',
  '11-02-24',
  'PAGO BIZUM A NATALIA;GRANDA;AL',
  '31,50',
  '5.558,63'],
 ['13-02-24', '980', '13-02-24', 'RECIBO /SIMYO', '23,43', '5.535,20'],
 ['13-02-24',
  '014',
  '11-02-24',
  'CAJERO LA CAIXA OF.6738',
  '80,00',
  '5.455,20'],
 ['13-02-24', '382', '11-02-24', 'COMISION MOV. DEB.', '2,95', '5.452,

In [121]:
def convert_transactions_to_DF(transacciones): 
    columnas = ['Fecha', 'Ref', 'Fecha Valor', 'Descripción', 'cargos_abonos', 'Saldo']
    df = pd.DataFrame(transacciones, columns=columnas)
    df['Fecha'] = pd.to_datetime(df['Fecha'], errors='coerce', format='%d-%m-%y')
    df['Fecha Valor'] = pd.to_datetime(df['Fecha Valor'], errors='coerce', format='%d-%m-%y')
    # Convertir las columnas de importe a numérico
    return df


In [122]:
df = convert_transactions_to_DF(transacciones)

In [123]:
df

,Fecha,Ref,Fecha Valor,Descripción,cargos_abonos,Saldo
0,2024-02-02,846,2024-02-01,RECIBO PLATINUM,"1.285,52","5.793,33"
1,2024-02-02,006,2024-02-01,PAGO BIZUM A NATALIA;GRANDA;AL,"4,40","5.788,93"
2,2024-02-07,134,2024-02-07,PAGO BIZUM A ROBERTO;PERINELLI,"85,00","5.703,93"
3,2024-02-10,201,2024-02-10,PAGO BIZUM A ANE;RAMSDEN;ARRIB,"33,00","5.670,93"
4,2024-02-10,322,2024-02-10,PAGO BIZUM A SAMIR RUBEN;AUYAN,"70,80","5.600,13"
5,2024-02-12,042,2024-02-12,PAGO BIZUM A DANIEL;TANO;LEAL,"5,00","5.595,13"
6,2024-02-12,505,2024-02-11,PAGO BIZUM A ANE;RAMSDEN;ARRIB,"5,00","5.590,13"
7,2024-02-12,662,2024-02-11,PAGO BIZUM A NATALIA;GRANDA;AL,"31,50","5.558,63"
8,2024-02-13,980,2024-02-13,RECIBO /SIMYO,"23,43","5.535,20"
9,2024-02-13,014,2024-02-11,CAJERO LA CAIXA OF.6738,"80,00","5.455,20"


In [114]:
len(transacciones[0])

6